> Instalação do framework e solver:

In [ ]:
!apt-get install -y -qq glpk-utils
!pip install -q pyomo

In [2]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np

pd.options.mode.chained_assignment = None

> Conexão com os dados do problema:

In [4]:
dados = "https://github.com/fersondahl-uff/DEA/raw/main/Dados-Distribuidores_barb.xlsx"

dados_df = pd.read_excel(io=dados)

dados_df = dados_df.rename(columns={'Distribuidora': 'DMU' ,'INPUT 1': 'Input 1',  'INPUT 2': 'Input 2',  'OUTPUT': 'Output'})

dados_df

,DMU,Input 1,Input 2,Output
0,A,39.975,1.54,3.35
1,B,40.880,1.55,5.52
2,C,41.990,1.19,6.83
3,D,44.510,3.43,19.53
4,E,52.915,2.07,10.93
5,F,56.305,2.21,14.61
6,G,57.485,2.63,6.99
7,H,59.195,2.82,9.12
8,I,98.635,2.13,8.36
9,J,100.930,4.41,12.92


# **2.**

> Resolução do problema utilizando o Método CCR do envelope Orientado à 
*Inputs*

In [5]:
## Modelo CCR envolope orientado à input
resultado = {'DMU':[], 'h': [], 'lambda': [], 'folgas': []}
n_input = 2

dmus = range(len(dados_df))
for problem in dmus:
    
    model = pyo.ConcreteModel()
    
    model.lambdas = pyo.Var(dmus, bounds=(0, np.inf))
    model.h_dmu = pyo.Var()
    
    model.slack = pyo.Var(range(3), bounds=(0, np.inf))
    
    
    lambdas = model.lambdas
    h_dmu = model.h_dmu
    folga = model.slack 
    
    model.C1 = pyo.ConstraintList()
    for j in range(1, n_input+1):
      model.C1.add(expr= sum(lambdas[i]*dados_df[f'Input {j}'][i] for i in dmus) - h_dmu * dados_df[f'Input {j}'][problem] + folga[j-1] ==0)
    
    model.C2 = pyo.Constraint(expr= sum(lambdas[i]*dados_df['Output'][i] for i in dmus) == dados_df['Output'][problem]  + folga[2] )
    
    
    model.obj = pyo.Objective(expr=h_dmu, sense=minimize)
    
    opt = SolverFactory('glpk')
    opt.solve(model)

    
    lambdas_result = []
    for i in dmus:
        lambdas_result.append(pyo.value(lambdas[i]))
    
    s_result = []
    for s in range(n_input + 1):
        s_result.append(pyo.value(folga[s]))    
    
    resultado['h'].append(pyo.value(h_dmu))
    resultado['lambda'].append(lambdas_result)
    resultado['folgas'].append(s_result)
    resultado['DMU'] = dados_df['DMU']

resultado_df = pd.DataFrame(data=resultado)

## Tratamento dos Resultados



lambda_list = []
column_name = []
for i in resultado_df.index:
    
    lambda_list.append(list(resultado_df.query(f'index == {i}')['lambda'].explode('lambda')))
    column_name.append(f'lambda_{i}')
    
lambdas_df = pd.DataFrame(lambda_list, columns=column_name)


folga_list = []
column_name = []
for i in resultado_df.index:
     folga_list.append(list(resultado_df.query(f'index == {i}')['folgas'].explode('folgas')))
     
for i in range(len(folga_list[0])):     
     column_name.append(f'folga_{i}')

  

resultado_df = pd.merge(left=resultado_df.drop(['lambda', 'folgas'], axis=1),
                        right=pd.merge(left=lambdas_df, right=pd.DataFrame(folga_list, columns=column_name), how='left', left_index=True, right_index=True),
                        how='left', left_index=True, right_index=True).set_index('DMU')

resultado_df

,h,lambda_0,lambda_1,lambda_2,lambda_3,lambda_4,lambda_5,lambda_6,lambda_7,lambda_8,lambda_9,lambda_10,lambda_11,folga_0,folga_1,folga_2
DMU,,,,,,,,,,,,,,,,
A,0.329053,0.0,0.0,0.0,0.000000,0.0,0.229295,0.0,0.0,0.0,0.0,0.0,0.0,2.434473e-01,0.0,0.0
B,0.538703,0.0,0.0,0.0,0.000000,0.0,0.377823,0.0,0.0,0.0,0.0,0.0,0.0,7.488338e-01,0.0,0.0
C,0.868192,0.0,0.0,0.0,0.000000,0.0,0.467488,0.0,0.0,0.0,0.0,0.0,0.0,1.013347e+01,0.0,0.0
D,1.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
E,0.798715,0.0,0.0,0.0,0.000000,0.0,0.748118,0.0,0.0,0.0,0.0,0.0,0.0,1.412388e-01,0.0,0.0
F,1.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,7.105427e-15,0.0,0.0
G,0.418859,0.0,0.0,0.0,0.093003,0.0,0.354117,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
H,0.514830,0.0,0.0,0.0,0.151903,0.0,0.421173,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
I,0.593702,0.0,0.0,0.0,0.000000,0.0,0.572211,0.0,0.0,0.0,0.0,0.0,0.0,2.634150e+01,0.0,0.0


**A.** 

> Através do método do envelope orientado à *input*, a eficiência de cada DMU será o valor da função objetivo após a otimização, ou o próprio valor de h.

In [6]:
eficiencia_df = resultado_df[['h']].apply(lambda linha: round(linha*100, 2)).rename(columns={'h': 'Eficiência %'})
eficiencia_df

,Eficiência %
DMU,
A,32.91
B,53.87
C,86.82
D,100.00
E,79.87
F,100.00
G,41.89
H,51.48
I,59.37


---

**B.**

> Os lambdas definirão os *benchmark* de cada DMU, sendo caracterizados em 3 casos:


1.   A DMU é eficiente:
  
  Ela será seu próprio *Benchmark*, seu respectivo lambda igual a 1 e os outros iguais a zero;

2.   A DMU é fracamente eficiente:
  
  Por mais que sua eficiência seja igual a 100%, ela terá outra DMU como *benchmark*;

3.   A DMU é ineficiente:
  
  Terá outra DMU ou uma combinação linear de 2 DMU's como *benchmark*.




In [7]:
lambda_dmu = { 0: 'A', 1: 'B', 2:'C', 3:'D',4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J', 10:'K', 11:'L'}


bench_df = resultado_df[['lambda_0', 'lambda_1', 'lambda_2', 'lambda_3', 'lambda_4',
       'lambda_5', 'lambda_6', 'lambda_7', 'lambda_8', 'lambda_9', 'lambda_10',
       'lambda_11']]

for i in range(12):
  bench_df[f'lambda_{i}'] = bench_df[f'lambda_{i}'].apply(lambda linha: lambda_dmu[i] if linha > 0 else '')

bench_df['benchmark'] = bench_df[['lambda_0', 'lambda_1', 'lambda_2', 'lambda_3', 'lambda_4',
       'lambda_5', 'lambda_6', 'lambda_7', 'lambda_8', 'lambda_9', 'lambda_10',
       'lambda_11']].agg(''.join, axis=1).apply(lambda linha: '-'.join(linha))

bench_df = pd.merge(left=bench_df[['benchmark']], right=eficiencia_df, how='left', right_index=True, left_index=True).reset_index()
bench_df['status_dmu'] = np.where(bench_df['DMU'] == bench_df['benchmark'], 'Eficiente', np.where(bench_df['Eficiência %'] == 100, 'Fracamente Eficiente', 'Ineficiente'))

bench_df

,DMU,benchmark,Eficiência %,status_dmu
0,A,F,32.91,Ineficiente
1,B,F,53.87,Ineficiente
2,C,F,86.82,Ineficiente
3,D,D,100.00,Eficiente
4,E,F,79.87,Ineficiente
5,F,F,100.00,Eficiente
6,G,D-F,41.89,Ineficiente
7,H,D-F,51.48,Ineficiente
8,I,F,59.37,Ineficiente
9,J,D-F,45.63,Ineficiente


---

**C.**

> O cálculo do alvo será feito em duas partes:


1.   Projeção Radial:

  Multiplicar o valor do *input* pelo fator de redução h;

2.   Alvo:

  Somar a projeção radial com a respectiva folga.

Para o *output*, basta somar seu valor inicial com sua folga.



In [8]:
alvos_df = pd.merge(left=dados_df.set_index('DMU'), right=resultado_df[['h', 'folga_0', 'folga_1', 'folga_2']], how='left', right_index=True, left_index=True)

for i in range(1, n_input+1):
  alvos_df[f'projecao radial ^X{i}'] = (alvos_df[f'Input {i}']*alvos_df['h'])
  alvos_df[f'alvo ^X{i}'] = (alvos_df[f'Input {i}']*alvos_df['h']) + alvos_df[f'folga_{i-1}']

alvos_df['^Y'] = alvos_df['Output'] + alvos_df[f'folga_{n_input}']

alvos_df

,Input 1,Input 2,Output,h,folga_0,folga_1,folga_2,projecao radial ^X1,alvo ^X1,projecao radial ^X2,alvo ^X2,^Y
DMU,,,,,,,,,,,,
A,39.975,1.54,3.35,0.329053,2.434473e-01,0.0,0.0,13.153902,13.397350,0.506742,0.506742,3.35
B,40.880,1.55,5.52,0.538703,7.488338e-01,0.0,0.0,22.022181,22.771015,0.834990,0.834990,5.52
C,41.990,1.19,6.83,0.868192,1.013347e+01,0.0,0.0,36.455384,46.588855,1.033149,1.033149,6.83
D,44.510,3.43,19.53,1.000000,0.000000e+00,0.0,0.0,44.510000,44.510000,3.430000,3.430000,19.53
E,52.915,2.07,10.93,0.798715,1.412388e-01,0.0,0.0,42.264007,42.405246,1.653340,1.653340,10.93
F,56.305,2.21,14.61,1.000000,7.105427e-15,0.0,0.0,56.305000,56.305000,2.210000,2.210000,14.61
G,57.485,2.63,6.99,0.418859,0.000000e+00,0.0,0.0,24.078115,24.078115,1.101599,1.101599,6.99
H,59.195,2.82,9.12,0.514830,0.000000e+00,0.0,0.0,30.475333,30.475333,1.451819,1.451819,9.12
I,98.635,2.13,8.36,0.593702,2.634150e+01,0.0,0.0,58.559826,84.901323,1.264586,1.264586,8.36


---

**D.**

> Resolução do problema utilizando o Método CCR de multiplicadores Orientado à 
*Inputs*

In [9]:
## MODELO CCR MULTIPLICADORES ORIENTADO À INPUT

dmus = range(len(dados_df))

resultado = {"DMU": [], "v1": [], "v2": [], "u": [], "eficiencia %": []}

for problem in dmus:
    
    
    model = pyo.ConcreteModel()
    
    model.inputs = pyo.Var(range(2), bounds=(0, np.inf))
    model.u = pyo.Var(bounds=(0, np.inf))
    u = model.u
    
    inputs = model.inputs
    
    model.C1 = pyo.Constraint(expr = dados_df['Input 1'][problem]*inputs[0] + dados_df['Input 2'][problem]*inputs[1] == 1 )
    
    model.C2 = pyo.ConstraintList()
    for dmu in dmus:
        model.C2.add(expr= dados_df['Output'][dmu]*u - dados_df['Input 1'][dmu]*inputs[0] - dados_df['Input 2'][dmu]*inputs[1] <= 0)
    
   

    model.obj = pyo.Objective(expr=dados_df['Output'][problem]*u , sense=maximize)
    
    opt = SolverFactory('glpk')
    opt.solve(model)

    resultado['v1'].append(pyo.value(inputs[0]))
    resultado['v2'].append(pyo.value(inputs[1]))
    resultado['u'].append(pyo.value(u))
    resultado['DMU'] = dados_df['DMU']
    resultado['eficiencia %'].append(round(100*pyo.value(model.obj),2))
    

resultado_mult_input = pd.DataFrame(data=resultado).set_index('DMU')

resultado_mult_input['status'] = np.where(resultado_mult_input['eficiencia %'] == 100, 'Eficiente', 'Ineficiente')

print("Tabela com os resultados do método dos Multilplicadores (Orientação: Input)")
resultado_mult_input


Tabela com os resultados do método dos Multilplicadores (Orientação: Input)


,v1,v2,u,eficiencia %,status
DMU,,,,,
A,0.000000,0.649351,0.098225,32.91,Ineficiente
B,0.000000,0.645161,0.097591,53.87,Ineficiente
C,0.000000,0.840336,0.127115,86.82,Ineficiente
D,0.003756,0.242805,0.051203,100.00,Eficiente
E,0.000000,0.483092,0.073075,79.87,Ineficiente
F,0.000000,0.452489,0.068446,100.00,Eficiente
G,0.004396,0.284152,0.059923,41.89,Ineficiente
H,0.004141,0.267688,0.056451,51.48,Ineficiente
I,0.000000,0.469484,0.071017,59.37,Ineficiente


---

# **3.**

> Resolução do problema utilizando o Método CCR do envelope Orientado à 
*Outputs*

In [10]:
## Modelo CCR envolope orientado à output

resultado = {'DMU':[], 'fi': [], 'lambda': [], 'folgas': []}
n_input = 2

dmus = range(len(dados_df))
for problem in dmus:
    
    model = pyo.ConcreteModel()
    
    model.lambdas = pyo.Var(dmus, bounds=(0, np.inf))
    model.fi = pyo.Var()
    
    model.slack = pyo.Var(range(3), bounds=(0, np.inf))
    
    
    lambdas = model.lambdas
    fi = model.fi
    folga = model.slack 
    

    model.C1 = pyo.ConstraintList()
    for j in range(1, n_input+1):
      model.C1.add(expr= sum(lambdas[i]*dados_df[f'Input {j}'][i] for i in dmus) - dados_df[f'Input {j}'][problem] + folga[j-1] ==0)
    
    model.C2 = pyo.Constraint(expr= sum(lambdas[i]*dados_df['Output'][i] for i in dmus) - dados_df['Output'][problem]*fi - folga[2] == 0)
    
    
    model.obj = pyo.Objective(expr=fi, sense=maximize)
    
    opt = SolverFactory('glpk')
    opt.solve(model)
    
    
    lambdas_result = []
    for i in dmus:
        lambdas_result.append(pyo.value(lambdas[i]))
    
    s_result = []
    for s in range(n_input +1):
        s_result.append(pyo.value(folga[s]))    
    
    resultado['fi'].append(pyo.value(fi))
    resultado['lambda'].append(lambdas_result)
    resultado['folgas'].append(s_result)
    resultado['DMU'] = dados_df['DMU']

resultado_out_df = pd.DataFrame(data=resultado)


lambda_list = []
column_name = []
for i in resultado_out_df.index:
    
    lambda_list.append(list(resultado_out_df.query(f'index == {i}')['lambda'].explode('lambda')))
    column_name.append(f'lambda_{i}')
    
lambdas_df = pd.DataFrame(lambda_list, columns=column_name)


folga_list = []
column_name = []
for i in resultado_out_df.index:
     folga_list.append(list(resultado_out_df.query(f'index == {i}')['folgas'].explode('folgas')))
     
for i in range(len(folga_list[0])):     
     column_name.append(f'folga_{i}')



resultado_out_df = pd.merge(left=resultado_out_df.drop(['lambda', 'folgas'], axis=1),
                        right=pd.merge(left=lambdas_df, right=pd.DataFrame(folga_list, columns=column_name), how='left', left_index=True, right_index=True),
                        how='left', left_index=True, right_index=True).set_index('DMU')

resultado_out_df

,fi,lambda_0,lambda_1,lambda_2,lambda_3,lambda_4,lambda_5,lambda_6,lambda_7,lambda_8,lambda_9,lambda_10,lambda_11,folga_0,folga_1,folga_2
DMU,,,,,,,,,,,,,,,,
A,3.039022,0.0,0.0,0.0,0.000000,0.0,0.696833,0.0,0.0,0.0,0.0,0.0,0.0,0.739842,0.000000e+00,0.0
B,1.856310,0.0,0.0,0.0,0.000000,0.0,0.701357,0.0,0.0,0.0,0.0,0.0,0.0,1.390068,0.000000e+00,0.0
C,1.151819,0.0,0.0,0.0,0.000000,0.0,0.538462,0.0,0.0,0.0,0.0,0.0,0.0,11.671923,0.000000e+00,0.0
D,1.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.440892e-16,0.0
E,1.252011,0.0,0.0,0.0,0.000000,0.0,0.936652,0.0,0.0,0.0,0.0,0.0,0.0,0.176833,0.000000e+00,0.0
F,1.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.0
G,2.387438,0.0,0.0,0.0,0.222040,0.0,0.845431,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.0
H,1.942391,0.0,0.0,0.0,0.295055,0.0,0.818082,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.0
I,1.684346,0.0,0.0,0.0,0.000000,0.0,0.963801,0.0,0.0,0.0,0.0,0.0,0.0,44.368190,0.000000e+00,0.0


**A.**

> As eficiências das DMU's não mudam com o modelo orientado à *output*, são exatamente as mesmas.

In [11]:
eficiencia_out_df = pd.DataFrame()
eficiencia_out_df['Eficiencia output %'] = resultado_out_df[['fi']].apply(lambda linha: round(100/linha, 2))
eficiencia_out_df = pd.merge(left=eficiencia_out_df, right=eficiencia_df, how='left', right_index=True, left_index=True)

eficiencia_out_df['Teste eficiencia'] = eficiencia_out_df['Eficiencia output %'] == eficiencia_out_df['Eficiência %']
eficiencia_out_df

,Eficiencia output %,Eficiência %,Teste eficiencia
DMU,,,
A,32.91,32.91,True
B,53.87,53.87,True
C,86.82,86.82,True
D,100.00,100.00,True
E,79.87,79.87,True
F,100.00,100.00,True
G,41.89,41.89,True
H,51.48,51.48,True
I,59.37,59.37,True


**B.**

> O cálculo do alvo será feito da seguinte maneira para cada caso:


1.   *Inputs*:

  Os *inputs* iniciais serão somados com suas respectivas folgas;

2.   *Output*:

  O *output* inicial de cada DMU será multiplicado por seu fi e somado com a sua folga.


In [12]:
lambda_dmu = { 0: 'A', 1: 'B', 2:'C', 3:'D',4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J', 10:'K', 11:'L'}


bench_out_df = resultado_out_df[['lambda_0', 'lambda_1', 'lambda_2', 'lambda_3', 'lambda_4',
       'lambda_5', 'lambda_6', 'lambda_7', 'lambda_8', 'lambda_9', 'lambda_10',
       'lambda_11']]

for i in range(12):
  bench_out_df[f'lambda_{i}'] = bench_out_df[f'lambda_{i}'].apply(lambda linha: lambda_dmu[i] if linha > 0 else '')

bench_out_df['benchmark'] = bench_out_df[['lambda_0', 'lambda_1', 'lambda_2', 'lambda_3', 'lambda_4',
       'lambda_5', 'lambda_6', 'lambda_7', 'lambda_8', 'lambda_9', 'lambda_10',
       'lambda_11']].agg(''.join, axis=1).apply(lambda linha: '-'.join(linha))

bench_out_df = pd.merge(left=bench_out_df[['benchmark']], right=eficiencia_df, how='left', right_index=True, left_index=True).reset_index()
bench_out_df['status_dmu'] = np.where(bench_out_df['DMU'] == bench_out_df['benchmark'], 'Eficiente', np.where(bench_out_df['Eficiência %'] == 100, 'Fracamente Eficiente', 'Ineficiente'))

alvos = []
alvos_out = pd.merge(left=dados_df.set_index('DMU'), right=resultado_out_df[['fi', 'folga_0', 'folga_1', 'folga_2']], how='left', right_index=True, left_index=True)

for i in range(1, n_input+1):
  alvos_out[f'alvo ^X{i}'] = alvos_out[f'Input {i}'] + alvos_out[f'folga_{i-1}']
  alvos.append(f'alvo ^X{i}')

alvos_out['alvo ^Y'] = alvos_out['Output']*alvos_out['fi'] + alvos_out[f'folga_{n_input}']
alvos.append('alvo ^Y')


bench_out_df = pd.merge(left=bench_out_df.set_index('DMU'), right=alvos_out[alvos], how='left', right_index=True, left_index=True)

bench_out_df

,benchmark,Eficiência %,status_dmu,alvo ^X1,alvo ^X2,alvo ^Y
DMU,,,,,,
A,F,32.91,Ineficiente,40.714842,1.54,10.180724
B,F,53.87,Ineficiente,42.270068,1.55,10.246833
C,F,86.82,Ineficiente,53.661923,1.19,7.866923
D,D,100.00,Eficiente,44.510000,3.43,19.530000
E,F,79.87,Ineficiente,53.091833,2.07,13.684480
F,F,100.00,Eficiente,56.305000,2.21,14.610000
G,D-F,41.89,Ineficiente,57.485000,2.63,16.688190
H,D-F,51.48,Ineficiente,59.195000,2.82,17.714602
I,F,59.37,Ineficiente,143.003190,2.13,14.081131


> Resolução do problema utilizando o Método CCR de multiplicadores Orientado à 
*outputs*

In [13]:
## MODELO CCR MULTIPLICADORES ORIENTADO À OUTPUT

dmus = range(len(dados_df))

resultado = {"DMU": [], "v1": [], "v2": [], "u": [], "eficiencia %": []}

for problem in dmus:
    
    
    model = pyo.ConcreteModel()
    
    model.inputs = pyo.Var(range(2), bounds=(0, np.inf))
    model.u = pyo.Var(bounds=(0, np.inf))
    u = model.u
    
    inputs = model.inputs
    
    model.C1 = pyo.Constraint(expr = dados_df['Output'][problem]*u == 1 )
    
    model.C2 = pyo.ConstraintList()
    for dmu in dmus:
        model.C2.add(expr= dados_df['Output'][dmu]*u - dados_df['Input 1'][dmu]*inputs[0] - dados_df['Input 2'][dmu]*inputs[1] <= 0)

    
    model.obj = pyo.Objective(expr= dados_df['Input 1'][problem]*inputs[0] + dados_df['Input 2'][problem]*inputs[1], sense=minimize)
    
    opt = SolverFactory('glpk')
    opt.solve(model)

    resultado['v1'].append(pyo.value(inputs[0]))
    resultado['v2'].append(pyo.value(inputs[1]))
    resultado['u'].append(pyo.value(u))
    resultado['DMU'] = dados_df['DMU']
    resultado['eficiencia %'].append(round(100/pyo.value(model.obj), 2))
    

resultado_mult_output = pd.DataFrame(data=resultado).set_index('DMU')

resultado_mult_output['status'] = np.where(resultado_mult_output['eficiencia %'] == 100, 'Eficiente', 'Ineficiente')

print("Tabela com os resultados do método dos Multilplicadores (Orientação: Output)")
resultado_mult_input

resultado_mult_output

Tabela com os resultados do método dos Multilplicadores (Orientação: Output)


,v1,v2,u,eficiencia %,status
DMU,,,,,
A,0.000000,1.973391,0.298507,32.91,Ineficiente
B,0.000000,1.197620,0.181159,53.87,Ineficiente
C,0.000000,0.967915,0.146413,86.82,Ineficiente
D,0.022467,0.000000,0.051203,100.00,Eficiente
E,0.000000,0.604836,0.091491,79.87,Ineficiente
F,0.005021,0.324571,0.068446,100.00,Eficiente
G,0.010494,0.678395,0.143062,41.89,Ineficiente
H,0.008043,0.519954,0.109649,51.48,Ineficiente
I,0.000000,0.790773,0.119617,59.37,Ineficiente
